In [6]:
#Se importan todas las librerias
import agentpy as ap
import socket
import numpy as np
from IPython.display import HTML
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
import random

M = 10
N = 10

In [7]:
# Se crea el Dummy Agent en la posicion incial de 0,0
class DummyAgent(ap.Agent):
  def setup(self):
    self.pos = (0,0)

  def execute(self):
    # Se saca la posición del agente
    posX, posY = self.model.enviroment.positions[self]

    # Se calcula un paso aleatorio en X y Y
    cambioX = random.randint(-1, 1)
    cambioY = random.randint(-1, 1)

    # Se camina a esa nueva posición.
    # Se usa la combinación min para que no salga del area de juego
    nuevaX = max(0, min(M-1, posX + cambioX))
    nuevaY = max(0, min(N-1, posY + cambioY))

    # Se mueve el agente en la cuadricula
    self.model.enviroment.move_to(self, (nuevaX, nuevaY))


In [3]:
class DummyModel(ap.Model):
  def setup(self):
    # Se crea un espacio M x N con un agente en la posición 0,0
    self.enviroment = ap.Grid(self,(M, N))
    self.agent = DummyAgent(self)
    self.agent.setup()
    self.enviroment.add_agents([self.agent], positions=[(M//2, N//2)])

    self.s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    self.s.connect(("127.0.0.1", 1101))

  def step(self):
    # Se ejecuta el paso del agente con cada frame
    self.agent.execute()

    # Posición actual del agente
    posX, posY = self.enviroment.positions[self.agent]

    mensaje = f"{posX},{posY}".encode("ascii")
    self.s.send(mensaje)

In [4]:
def my_plot(model, ax):
    ax.clear()

    # Construimos la grilla de visualización en cada frame
    view_grid = np.zeros((M, N), dtype=int)

    # Posición del agente
    posX, posY = model.enviroment.positions[model.agent]
    view_grid[posX, posY] = 1

    # Mapa de color simple: 0 = blanco, 1 = gris oscuro
    cmap = ListedColormap([(1, 1, 1, 1), (0.2, 0.2, 0.2, 1)])

    ax.imshow(view_grid, cmap=cmap, vmin=0, vmax=1, origin='lower')
    ax.set_aspect('equal')

    # Cuadrícula y ejes “vacíos”
    ax.set_xlim(-0.5, N - 0.5)
    ax.set_ylim(-0.5, M - 0.5)
    ax.set_xticks(np.arange(-0.5, N, 1))
    ax.set_yticks(np.arange(-0.5, M, 1))
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.grid(True, linewidth=0.5)

In [10]:
# Se ejecuta todo y se dibuja
fig, ax = plt.subplots()
parameters = {'print': False, 'steps': 50}
dummyModel = DummyModel(parameters)
animation = ap.animate(dummyModel, fig, ax, my_plot)
HTML(animation.to_jshtml())